In [1]:
import os

In [3]:
%pwd

'c:\\Users\\USER\\Documents\\GitHub\\End-to-End-Customer-Churn-Prediction-using-MLflow-and-DVC'

In [2]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\USER\\Documents\\GitHub\\End-to-End-Customer-Churn-Prediction-using-MLflow-and-DVC'

In [5]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/Oyebamiji-Micheal/End-to-End-Customer-Churn-Prediction-using-MLflow-and-DVC.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="Oyebamiji-Micheal"
os.environ["MLFLOW_TRACKING_PASSWORD"]="c2504ce48d7aac171c95a633559cfc197c566b8c"

In [26]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [27]:
from mlFlowProject.constants import *
from mlFlowProject.utils.common import read_yaml, create_directories, save_json

In [28]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.LightGBM
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri="https://dagshub.com/Oyebamiji-Micheal/End-to-End-Customer-Churn-Prediction-using-MLflow-and-DVC.mlflow",  
        )

        return model_evaluation_config

In [29]:
import os
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [33]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
    
    def eval_metrics(self, actual, pred):
        accuracy = accuracy_score(actual, pred)
        precision = precision_score(actual, pred)
        recall = recall_score(actual, pred)
        f1 = f1_score(actual, pred)

        return accuracy, precision, recall, f1
    

    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():

            predictions = model.predict(test_x)

            (accuracy, precision, recall, f1) = self.eval_metrics(test_y, predictions)
            
            # Saving metrics as local
            scores = {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("accuracy", accuracy)
            mlflow.log_metric("precision", precision)
            mlflow.log_metric("recall", recall)
            mlflow.log_metric("f1-score", f1)

            # Model registry does not work with file store
            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(model, "model", registered_model_name="LightGBM")
            else:
                mlflow.sklearn.log_model(model, "model")


In [34]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2024-05-26 10:33:13,533: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-05-26 10:33:13,537: INFO: common: yaml file: params.yaml loaded successfully]
[2024-05-26 10:33:13,543: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-05-26 10:33:13,545: INFO: common: created directory at: artifacts]
[2024-05-26 10:33:13,547: INFO: common: created directory at: artifacts/model_evaluation]
https
[2024-05-26 10:33:17,764: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]
[2024-05-26 10:35:29,735: WARNING: connectionpool: Retrying (Retry(total=4, connect=5, read=4, redirect=5, status=5)) after connection broken by 'ReadTimeoutError("HTTPSConnectionPool(host='dagshub.com', port=443): Read timed out. (read timeout=120)")': /Oyebamiji-Micheal/End-to-End-Customer-Churn-Prediction-using-MLflow-and-DVC.mlflow/api/2.0/mlflow-artifacts/artifacts/55dd08b06e8545cbb5bd675525641ebe/1dab9afb936b4cefbf70e5696ad69208/artifacts/model/model.pkl]
[2024

KeyboardInterrupt: 